In [148]:
import streamlit as st
import sqlite3 as sq
import requests as rq
import pandas as pd

import folium
import json
import os

Запросы для маршрутов

In [149]:
routes_from_to_url = 'http://irkbus.ru/php/getRoutes.php?city=irkutsk&info=12345&lang=ru&_=1686163028764'
stations_url = 'http://irkbus.ru/php/getStations.php?city=irkutsk&info=12345&lang=ru&_=1686163028767'
transport_tramvay_2_url = 'http://irkbus.ru/php/getVehicleForecasts.php?vid=868204005990253&type=0&city=irkutsk&info=12345&lang=ru&_=1686171778553'      # Если поймем, откуда брать vid, то это будет победа

In [157]:
stations_data[stations_data['id'] == 821]

,id,name,descr,lat,lng,type
520,821,СНТ Багульник,в центр,52.272479,104.540013,0


In [150]:
response = rq.get(stations_url)
stations_data = pd.DataFrame.from_dict(json.loads(response.text) if response.status_code == 200 else False).sort_values(by = 'id')

In [151]:
response = rq.get(routes_from_to_url)
routes_from_to_data = pd.DataFrame.from_dict(json.loads(response.text) if response.status_code == 200 else False).replace('Тр', 'Трамвай').replace('А', 'Автобус').replace('Т', 'Троллейбус').sort_values(by = 'id').reset_index().drop('index', axis = 1)

In [152]:
class Routes():
    def __init__(self,  routes_df : pd.DataFrame, stations_df : pd.DataFrame):
        self.routes_df =  routes_df
        self.stations_df =  stations_df


        self.routes_names_constraints = {'Avtobus_' : 'Автобус', ' Троллейбус' : 'Троллейбус'}

        self.routes_df.name = self.routes_df.name.apply(self.routes_names_correcter)
        stations_df.lat = stations_df.lat.apply(self.stations_cords_correcter)
        stations_df.lng = stations_df.lng.apply(self.stations_cords_correcter)

    def routes_names_correcter(self, name : str) -> str:
        for constraint in self.routes_names_constraints:
            if constraint in name:
                if constraint == None:
                    return('НЕ НАЙДЕНО')
                
                name = list(name)
                for let in constraint:
                    name.remove(let)
                
                name = ''.join(name) + f'_{self.routes_names_constraints[constraint]}'

        return(name)
    
    def stations_cords_correcter(self, cords : int) -> float:
        cords = str(cords)
        cords = list(cords)

        if '.' not in cords:
            if cords[0] == '1':
                cords.insert(3, '.')
                cords = float(''.join(cords))
            
            else:
                cords.insert(2, '.')
                cords = float(''.join(cords))
        
        return(cords)



    def route_data(self, route_name : str) -> pd.DataFrame:
        try:
            return(self.routes_df[self.routes_df['name'] == route_name])
        
        except:
            return(False)
        
    
    # def get_stations(self, route_name : str) -> list:
    #     # print(self.routes_df[self.routes_df['name'] == route_name][['id', 'name', 'type', 'num', 'fromst', 'fromstid', 'tost', 'tostid']])
    #     try:
    #         return(self.routes_df[['id', 'name', 'type', 'num', 'fromst', 'fromstid', 'tost', 'tostid'], self.routes_df['name'] == route_name])
        
    #     except:
    #         return(False)

---

In [153]:
routes_manipulator = Routes(routes_from_to_data, stations_data)

In [154]:
routes_manipulator.route_data('1Усолье')

,id,name,type,num,fromst,fromstid,tost,tostid
22,13,1Усолье,Трамвай,1,Трампарк,137,9 столовая,133
24,14,1Усолье,Трамвай,1,9 столовая,133,Трампарк,128


In [155]:
routes_manipulator.stations_df

,id,name,descr,lat,lng,type
826,1,Степана Разина,в сторону ж/д вокзала,52.283020,104.277680,1
44,1,Аэропорт,конечная 4 троллейбуса,52.273956,104.355826,0
503,2,Сквер им. Кирова,в сторону ул. Карла Маркса,52.285909,104.282473,0
825,2,Степана Разина,в сторону центрального рынка,52.283472,104.277002,1
748,3,Художественный музей,в сторону пл. Кирова,52.283113,104.280979,0
...,...,...,...,...,...,...
603,1028,СНТ Радуга-2,из города,52.102024,104.343842,0
604,1029,СНТ Радуга-2,в город,52.101687,104.344614,0
576,1030,СНТ Минерал,из города,52.093745,104.352436,0
577,1031,СНТ Минерал,в город,52.093350,104.353004,0
